In [23]:
shhh <- suppressPackageStartupMessages

# A function to collect fundamental data from Yahoo finance (Income Statement, Balance Sheet and Cash Flow statements)

getFin <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
    }else{
      install.packages("rvest")
      library(rvest)
      }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        # Collect the Income Statement data
        link <- "https://finance.yahoo.com/quote/"
        link <- paste0(link, stock[i], "/financials?p=", stock[i])
        wahis.session <- html_session(link)
        p <- wahis.session %>%
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/table')%>%
          html_table(fill = TRUE)
        IncomeStatement <- p[[1]]
        colnames(IncomeStatement) <- paste(IncomeStatement[1,])
        IncomeStatement <- IncomeStatement[-c(1,5,12,20,25),]
        names_row <- paste(IncomeStatement[,1])
        IncomeStatement <- IncomeStatement[,-1]
        IncomeStatement <- apply(IncomeStatement, 2, function(x){gsub(",","",x)})
        IncomeStatement <- as.data.frame(apply(IncomeStatement, 2, as.numeric))
        rownames(IncomeStatement) <- paste(names_row)
        temp1 <- IncomeStatement
        
        # Collect the Balance Sheet data
        link <- "https://finance.yahoo.com/quote/"
        link <- paste0(link, stock[i],"/balance-sheet?p=", stock[i])
        wahis.session <- html_session(link)
        p <- wahis.session %>%
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/table')%>%
          html_table(fill = TRUE)
        BalanceSheet <- p[[1]]
        colnames(BalanceSheet) <- BalanceSheet[1,]
        BalanceSheet <- BalanceSheet[-c(1,2,17,28),]
        names_row <- BalanceSheet[,1]
        BalanceSheet <- BalanceSheet[,-1]
        BalanceSheet <- apply(BalanceSheet, 2, function(x){gsub(",","",x)})
        BalanceSheet <- as.data.frame(apply(BalanceSheet, 2, as.numeric))
        rownames(BalanceSheet) <- paste(names_row)
        temp2 <- BalanceSheet
        
        # Collect the Cash Flow data
        link <- "https://finance.yahoo.com/quote/"
        link <- paste0(link, stock[i], "/cash-flow?p=", stock[i])
        wahis.session <- html_session(link)
        p <- wahis.session %>%
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/table')%>%
          html_table(fill = TRUE)
        CashFlow <- p[[1]]
        colnames(CashFlow) <- CashFlow[1,]
        CashFlow <- CashFlow[-c(1,3,11,16),]
        names_row <- CashFlow[,1]
        CashFlow <- CashFlow[,-1]
        CashFlow <- apply(CashFlow, 2, function(x){gsub(",","",x)})
        CashFlow <- as.data.frame(apply(CashFlow, 2, as.numeric))
        rownames(CashFlow) <- paste(names_row)
        temp3 <- CashFlow
        
        assign(paste0(stock[i],'.f'),value = list(IncomeStatement = temp1, BalanceSheet = temp2, CashFlow = temp3), envir = parent.frame())
        },
      error = function(cond){
        message(stock[i], "Give error ",cond)
        }
      )
  }
}

In [18]:
# Select the tickers we wish to collect
symbols <- c("GOOG", "MSFT", "HOG")

# Apply the function
suppressWarnings(tryCatch(getFin(symbols)))

symbols.f <- sapply(symbols, function(x) { paste0(x, ".f") })

tickers <- list2env(mget(symbols.f))

IS <- lapply(tickers, "[[", "IncomeStatement")
BS <- lapply(tickers, "[[", "BalanceSheet")
CF <- lapply(tickers, "[[", "CashFlow")

IS <- as.data.frame(IS)
BS <- as.data.frame(BS)
CF <- as.data.frame(CF)

head(IS)
head(BS)
head(CF)

HOG.f.12.31.2018 HOG.f.12.31.2017
Total Revenue                      5716875          5647224         
Cost of Revenue                    3803576          3739199         
Gross Profit                       1913299          1908025         
Research Development                    NA               NA         
Selling General and Administrative 1090436          1016761         
Non Recurring                           NA               NA         
                                   HOG.f.12.31.2016 HOG.f.12.31.2015
Total Revenue                      5996458          5995402         
Cost of Revenue                    3885527          3772841         
Gross Profit                       2110931          2222561         
Research Development                    NA               NA         
Selling General and Administrative 1061995          1040266         
Non Recurring                           NA               NA         
                                   MSFT.f.6.30.2019 MSFT.f.6.30.2018
Total Revenue                      125843000        110360000       
Cost of Revenue                     42910000         38353000       
Gross Profit                        82933000         72007000       
Research Development                16876000         14726000       
Selling General and Administrative  23098000         22223000       
Non Recurring                             NA               NA       
                                   MSFT.f.6.30.2017 MSFT.f.6.30.2016
Total Revenue                      96571000         91154000        
Cost of Revenue                    34261000         32780000        
Gross Profit                       62310000         58374000        
Research Development               13037000         11988000        
Selling General and Administrative 19942000         19198000        
Non Recurring                            NA               NA        
                                   GOOG.f.12.31.2018 GOOG.f.12.31.2017
Total Revenue                      136819000         110855000        
Cost of Revenue                     59549000          45583000        
Gross Profit                        77270000          65272000        
Research Development                21419000          16625000        
Selling General and Administrative  24459000          19765000        
Non Recurring                             NA                NA        
                                   GOOG.f.12.31.2016 GOOG.f.12.31.2015
Total Revenue                      90272000          74989000         
Cost of Revenue                    35138000          28164000         
Gross Profit                       55134000          46825000         
Research Development               13948000          12282000         
Selling General and Administrative 17470000          15183000         
Non Recurring                            NA                NA

HOG.f.12.31.2018 HOG.f.12.31.2017 HOG.f.12.31.2016
Cash And Cash Equivalents  544548           338186           425540         
Short Term Investments      10007               NA             5519         
Net Receivables            306474           329986           285106         
Inventory                  556128           538202           499917         
Other Current Assets       124168           156253           155191         
Total Current Assets      4484442          3884742          3853852         
                          HOG.f.12.31.2015 MSFT.f.6.30.2019 MSFT.f.6.30.2018
Cash And Cash Equivalents  400443           11356000         11946000       
Short Term Investments      45192          122476000        121718000       
Net Receivables            247405           29524000         26481000       
Inventory                  585907            2063000          2662000       
Other Current Assets       171171           10133000          6855000       
Total Current Assets      3977883          175552000        169662000       
                          MSFT.f.6.30.2017 MSFT.f.6.30.2016 GOOG.f.12.31.2018
Cash And Cash Equivalents   7663000          6510000         16701000        
Short Term Investments    125238000        106531000         92439000        
Net Receivables            22431000         18277000         21193000        
Inventory                   2181000          2251000          1107000        
Other Current Assets        5183000          6091000          4236000        
Total Current Assets      162696000        139660000        135676000        
                          GOOG.f.12.31.2017 GOOG.f.12.31.2016 GOOG.f.12.31.2015
Cash And Cash Equivalents  10715000          12918000         15409000         
Short Term Investments     91156000          73415000         56517000         
Net Receivables            18705000          15632000         13459000         
Inventory                    749000            268000           491000         
Other Current Assets        2983000           3175000          1590000         
Total Current Assets      124308000         105408000         90114000

HOG.f.12.31.2018 HOG.f.12.31.2017
Net Income                      531451           521759          
Depreciation                    230209           222188          
Adjustments To Net Income       150035           226144          
Changes In Accounts Receivables   9143           -18149          
Changes In Liabilities          196192            10128          
Changes In Inventories          -31059           -20584          
                                HOG.f.12.31.2016 HOG.f.12.31.2015
Net Income                      692164            752207         
Depreciation                    209555            198074         
Adjustments To Net Income       131961             54670         
Changes In Accounts Receivables -45934            -13665         
Changes In Liabilities           38237            138823         
Changes In Inventories           85072           -155222         
                                MSFT.f.6.30.2019 MSFT.f.6.30.2018
Net Income                      39240000         16571000        
Depreciation                    11600000          9900000        
Adjustments To Net Income       -2521000         -3054000        
Changes In Accounts Receivables -2812000         -3862000        
Changes In Liabilities           4694000          7070000        
Changes In Inventories            597000          -465000        
                                MSFT.f.6.30.2017 MSFT.f.6.30.2016
Net Income                      25489000         20539000        
Depreciation                     7800000          5878000        
Adjustments To Net Income        1342000          6229000        
Changes In Accounts Receivables -1216000           562000        
Changes In Liabilities           3901000          2653000        
Changes In Inventories             50000           600000        
                                GOOG.f.12.31.2018 GOOG.f.12.31.2017
Net Income                      30736000          12662000         
Depreciation                     9029000           6899000         
Adjustments To Net Income        3298000           8284000         
Changes In Accounts Receivables -2169000          -3768000         
Changes In Liabilities           1438000           1121000         
Changes In Inventories                NA                NA         
                                GOOG.f.12.31.2016 GOOG.f.12.31.2015
Net Income                      19478000          16348000         
Depreciation                     6100000           5024000         
Adjustments To Net Income        7158000           5609000         
Changes In Accounts Receivables -2578000          -2094000         
Changes In Liabilities            333000            246000         
Changes In Inventories                NA                NA

In [19]:
# Clean the data up a little bit
ISBSCF <- rbind(IS, BS, CF)

library(tibble)
library(tidyr)

ISBSCF <- ISBSCF %>%
  t() %>%
  data.frame() %>%
  rownames_to_column('rn') %>%
  separate(rn, into = c("symbol", "year"),
           sep = -4, convert = TRUE)


ISBSCF$symbol <- gsub("(f).*", "", ISBSCF$symbol)
ISBSCF$symbol <- gsub('.$', '', ISBSCF$symbol)
ISBSCF$symbol <- gsub('^X', '', ISBSCF$symbol)

head(ISBSCF)

symbol year Total.Revenue Cost.of.Revenue Gross.Profit Research.Development
1 HOG    2018   5716875      3803576         1913299           NA            
2 HOG    2017   5647224      3739199         1908025           NA            
3 HOG    2016   5996458      3885527         2110931           NA            
4 HOG    2015   5995402      3772841         2222561           NA            
5 MSFT   2019 125843000     42910000        82933000     16876000            
6 MSFT   2018 110360000     38353000        72007000     14726000            
  Selling.General.and.Administrative Non.Recurring Others
1  1090436                           NA            NA    
2  1016761                           NA            NA    
3  1061995                           NA            NA    
4  1040266                           NA            NA    
5 23098000                           NA            NA    
6 22223000                           NA            NA    
  Total.Operating.Expenses ... Investments
1  4894012                 ...  -10007    
2  4755960                 ...    6916    
3  4947522                 ...   40014    
4  4813107                 ...   11507    
5 82884000                 ...  540000    
6 75302000                 ... 6557000    
  Other.Cash.flows.from.Investing.Activities
1 -11598                                    
2    547                                    
3    411                                    
4   7474                                    
5     NA                                    
6 -98000                                    
  Total.Cash.Flows.From.Investing.Activities Dividends.Paid
1   -662269                                    -245810     
2   -562468                                    -251862     
3   -392731                                    -252321     
4   -915848                                    -249262     
5 -15773000                                  -13811000     
6  -6061000                                  -12699000     
  Sale.Purchase.of.Stock Net.Borrowings
1 NA                        618128     
2 NA                        155511     
3 NA                        -78256     
4 NA                       1397161     
5 NA                      -4000000     
6 NA                     -10201000     
  Other.Cash.Flows.from.Financing.Activities
1      NA                                   
2    8458                                   
3    2251                                   
4   14878                                   
5 -675000                                   
6 -971000                                   
  Total.Cash.Flows.From.Financing.Activities Effect.Of.Exchange.Rate.Changes
1    -14763                                   -15351                        
2   -550261                                    26747                        
3   -777885                                    -9443                        
4   -354064                                   -14677                        
5 -36887000                                  -115000                        
6 -33590000                                    50000                        
  Change.In.Cash.and.Cash.Equivalents
1  513538                            
2  -80921                            
3   -5720                            
4 -184471                            
5 -590000                            
6 4283000

In [20]:
############################## Get stats data ##################################
library(quantmod)
metrics <- yahooQF(c("Name", "Enterprise Value", "Enterprise Value/Revenue", "Enterprise Value/EBITDA",
                     "Total Cash Per Share (mrq)", "EBITDA", "Volume", "P/E Ratio", "Dividend Yield", "Shares Outstanding", "Price/Book"))

stats <- getQuote(symbols, what = metrics) %>%
  rownames_to_column("symbol")

colnames(stats) <- gsub(" ", ".", colnames(stats))
colnames(stats) <- gsub("/", ".", colnames(stats))

head(stats)
#################################################################################

symbol Trade.Time          Name                  Volume   P.E.Ratio
1 GOOG   2019-08-23 16:00:01 Alphabet Inc.          1688271 23.24195 
2 MSFT   2019-08-23 16:00:01 Microsoft Corporation 32523998 26.36166 
3 HOG    2019-08-23 16:02:58 Harley-Davidson, Inc.  1849495 11.37356 
  Dividend.Yield Shares.Outstanding Price.Book
1         NA      348264000         4.157572  
2 0.01335462     7662819840         9.962656  
3 0.04660619      159072992         2.489465

In [24]:
############################## Get latest stock price data ######################
shhh(library(tidyquant))
stocks <- symbols %>%
  tq_get(get = "stock.prices",
         from =  Sys.Date(),
         to =  Sys.Date()) %>%
  select(symbol, date, adjusted)

head(stocks)
#################################################################################

symbol date       adjusted
1 GOOG   2019-08-23 1151.29 
2 MSFT   2019-08-23  133.39 
3 HOG    2019-08-23   30.72

In [27]:
############################## Join all 3 datasets together ######################
financials <- ISBSCF %>%
  filter(year == 2018) %>%
  inner_join(stats, by = "symbol") %>%
  inner_join(stocks, by = "symbol")

head(financials)
#################################################################################

symbol year Total.Revenue Cost.of.Revenue Gross.Profit Research.Development
1 HOG    2018   5716875      3803576         1913299           NA            
2 MSFT   2018 110360000     38353000        72007000     14726000            
3 GOOG   2018 136819000     59549000        77270000     21419000            
  Selling.General.and.Administrative Non.Recurring Others
1  1090436                           NA            NA    
2 22223000                           NA            NA    
3 24459000                           NA            NA    
  Total.Operating.Expenses ... Change.In.Cash.and.Cash.Equivalents
1   4894012                ...  513538                            
2  75302000                ... 4283000                            
3 105427000                ... 5986000                            
  Trade.Time          Name                  Volume   P.E.Ratio Dividend.Yield
1 2019-08-23 16:02:58 Harley-Davidson, Inc.  1849495 11.37356  0.04660619    
2 2019-08-23 16:00:01 Microsoft Corporation 32523998 26.36166  0.01335462    
3 2019-08-23 16:00:01 Alphabet Inc.          1688271 23.24195          NA    
  Shares.Outstanding Price.Book date       adjusted
1  159072992         2.489465   2019-08-23   30.72 
2 7662819840         9.962656   2019-08-23  133.39 
3  348264000         4.157572   2019-08-23 1151.29

In [35]:
############################## Create some financial ratios ######################

financials %>%
  mutate(BvE = Total.Assets - Total.Liabilities,
         BvE_per_share = BvE / (Shares.Outstanding / 1000),
         EBITDA = Earnings.Before.Interest.and.Taxes + Depreciation,
         Market_Cap = adjusted * (Shares.Outstanding / 1000),                 #¡Cuidado!
         EV = (Market_Cap + Total.Liabilities) - Cash.And.Cash.Equivalents,
         Mkt_Book = EV /(Total.Assets - Cash.And.Cash.Equivalents),
         Price_Book = Market_Cap / Total.stockholders..equity,
         Price_Sales = Market_Cap / Total.Revenue,
         PE_Ratio = Market_Cap / Net.Income,
         EV_EBITDA =  EV / EBITDA,
         NI_REV = Net.Income / Total.Revenue,
         Rev_TA = Total.Revenue / Total.Assets,
         TA_BvE = Total.Assets / BvE,
         ROE = NI_REV * Rev_TA * TA_BvE * 100) %>%
  select(symbol, date, BvE, BvE_per_share, EBITDA, Market_Cap, EV, Mkt_Book, Price_Book, Price_Sales,
         PE_Ratio, EV_EBITDA, NI_REV, Rev_TA, TA_BvE, ROE)
#################################################################################

symbol date       BvE       BvE_per_share EBITDA   Market_Cap EV        
1 HOG    2019-08-23   1773949  11.15179      1053072    4886722   13233889
2 MSFT   2019-08-23  82718000  10.79472     44958000 1022143531 1186327531
3 GOOG   2019-08-23 177628000 510.03836     40421000  400952874  439415874
  Mkt_Book Price_Book Price_Sales PE_Ratio  EV_EBITDA NI_REV    Rev_TA   
1 1.307552  2.754714  0.8547891    9.195057 12.56694  0.0929618 0.5360074
2 4.804852 12.356966  9.2619022   61.682670 26.38746  0.1501540 0.4263506
3 2.033476  2.257262  2.9305350   13.045057 10.87098  0.2246472 0.5877307
  TA_BvE   ROE     
1 6.012385 29.95864
2 3.129283 20.03312
3 1.310559 17.30358

In [34]:
financials %>%
  ggplot(aes(x = symbol)) +
  geom_col(aes(y = Total.Assets, color = "blue", fill = "blue", alpha = 0.5)) +
  geom_col(aes(y = Total.Liabilities, color = "red", fill = "red", alpha = 0.5)) +
  ggtitle("Total Assets & Total Liabilities") +
  xlab("Symbols") +
  ylab("Total Assets & Total Liabilities") +
  theme_tq() +
  theme(legend.position = "none")